## Introdução

Neste notebook, iremos detalhar a construção de um Grafo (Rede) que represente a malha aérea brasileira.

Ele será construído com auxílio de dados públicos disponibilizados pela ANAC sobre os voos que aconteceram em aeroportos brasileiros.

Partiremos dos dados estruturados (tabelas .csv) e, com auxílio das bibliotecas Pandas e NetworkX, os transformaremos em um grafo.

Esta base será utilizada em outro *Jupyter Notebook* onde focaremos em estudar os aspectos matemáticos e estatísticos da rede criada.

Este exercício tem intuito puramente acadêmico.

## Importando dados dos aeroportos

Dados com todos os aeroportos do mundo (tabela com as coordenadas dos aeroportos)

https://drive.google.com/file/d/1ZOjvbNo083YfJpOpdCy3CiHMEzppIRQB/view?usp=sharing



Dados com todos os vôos com origem ou destino no Brasil (dados públicos da ANAC)

https://drive.google.com/file/d/13TdnsCxmsFxvDlWV2FJ_81GwKo4IVS0z/view?usp=sharing

https://www.anac.gov.br/assuntos/setor-regulado/empresas/envio-de-informacoes/base-de-dados-estatisticos-do-transporte-aereo

https://www.anac.gov.br/assuntos/dados-e-estatisticas/descricao-de-variaveis

In [1]:
!gdown https://drive.google.com/uc?id=1ZOjvbNo083YfJpOpdCy3CiHMEzppIRQB
!unzip airport-codes_zip.zip

Downloading...
From: https://drive.google.com/uc?id=1ZOjvbNo083YfJpOpdCy3CiHMEzppIRQB
To: /content/airport-codes_zip.zip
6.96MB [00:00, 19.1MB/s]
Archive:  airport-codes_zip.zip
  inflating: archive/airport-codes.csv  
  inflating: data/validation_report.json  
  inflating: data/airport-codes_csv.csv  
  inflating: data/airport-codes_json.json  
  inflating: README.md               
  inflating: datapackage.json        


In [2]:
!gdown https://drive.google.com/uc?id=13TdnsCxmsFxvDlWV2FJ_81GwKo4IVS0z
!unzip Dados_Estatisticos.zip

Downloading...
From: https://drive.google.com/uc?id=13TdnsCxmsFxvDlWV2FJ_81GwKo4IVS0z
To: /content/Dados Estatisticos.csv
108MB [00:00, 139MB/s]
unzip:  cannot find or open Dados_Estatisticos.zip, Dados_Estatisticos.zip.zip or Dados_Estatisticos.zip.ZIP.


## Pré processando a base de dados

Nesta etapa, faremos todo o pré processamento necessário para a construção do nosso Grafo.

**Importando o conjunto de dados**

In [254]:
import pandas as pd
import numpy as np

dados_anac = pd.read_csv('Dados_estatisticos.csv', sep=';', encoding='latin-1')

In [255]:
dados_anac.head()

,EMPRESA (SIGLA),EMPRESA (NOME),EMPRESA (NACIONALIDADE),ANO,MÊS,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE ORIGEM (NOME),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE ORIGEM (PAÍS),AEROPORTO DE ORIGEM (CONTINENTE),AEROPORTO DE DESTINO (SIGLA),AEROPORTO DE DESTINO (NOME),AEROPORTO DE DESTINO (UF),AEROPORTO DE DESTINO (REGIÃO),AEROPORTO DE DESTINO (PAÍS),AEROPORTO DE DESTINO (CONTINENTE),NATUREZA,GRUPO DE VOO,PASSAGEIROS PAGOS,PASSAGEIROS GRÁTIS,CARGA PAGA (KG),CARGA GRÁTIS (KG),CORREIO (KG),ASK,RPK,ATK,RTK,COMBUSTÍVEL (LITROS),DISTÂNCIA VOADA (KM),DECOLAGENS,CARGA PAGA KM,CARGA GRATIS KM,CORREIO KM,ASSENTOS,PAYLOAD,HORAS VOADAS,BAGAGEM (KG)
0,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2011,1,KAUS,"AUSTIN, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,INTERNACIONAL,REGULAR,0.0,0.0,0.0,0.0,0.0,75276.0,50184.0,13770.0,7463.0,NaN,306.0,1.0,2.946780e+06,0.0,0.0,246.0,45000.0,"2,5",NaN
1,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2011,1,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGL,RIO DE JANEIRO,RJ,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,2391.0,51.0,28903.0,0.0,6362.0,23968900.0,20129800.0,3830640.0,2108580.0,NaN,109447.0,13.0,2.433340e+08,0.0,53561700.0,2847.0,455000.0,"136,88",NaN
2,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2011,1,KDFW,"DALLAS & FORT WORTH, TEXAS",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGR,GUARULHOS,SP,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,6879.0,95.0,298598.0,0.0,9302.0,60840700.0,56710500.0,11129400.0,7642250.0,NaN,247320.0,30.0,2.461640e+09,0.0,76685700.0,7380.0,1350000.0,"299,5",NaN
3,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2011,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGL,RIO DE JANEIRO,RJ,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,REGULAR,5130.0,57.0,47901.0,0.0,41232.0,47394200.0,38274000.0,7806290.0,4130990.0,NaN,216412.0,28.0,3.701190e+08,0.0,316224000.0,6132.0,1010000.0,"285,26",NaN
4,AAL,"AMERICAN AIRLINES, INC.",ESTRANGEIRA,2011,1,KJFK,"NEW YORK, NEW YORK",NaN,NaN,ESTADOS UNIDOS DA AMÉRICA,AMÉRICA DO NORTE,SBGR,GUARULHOS,SP,SUDESTE,BRASIL,AMÉRICA DO SUL,INTERNACIONAL,NÃO REGULAR,219.0,1.0,6475.0,0.0,0.0,1885340.0,1678420.0,344880.0,200681.0,NaN,7664.0,1.0,4.962440e+07,0.0,0.0,246.0,45000.0,"10,22",NaN


In [256]:
print( f" Tamanho do dataset {len(dados_anac)}" )

 Tamanho do dataset 431250


**Visualizando as colunas e seus tipos**

In [257]:
dados_anac.dtypes

EMPRESA (SIGLA)                       object
EMPRESA (NOME)                        object
EMPRESA (NACIONALIDADE)               object
ANO                                    int64
MÊS                                    int64
AEROPORTO DE ORIGEM (SIGLA)           object
AEROPORTO DE ORIGEM (NOME)            object
AEROPORTO DE ORIGEM (UF)              object
AEROPORTO DE ORIGEM (REGIÃO)          object
AEROPORTO DE ORIGEM (PAÍS)            object
AEROPORTO DE ORIGEM (CONTINENTE)      object
AEROPORTO DE DESTINO (SIGLA)          object
AEROPORTO DE DESTINO (NOME)           object
AEROPORTO DE DESTINO (UF)             object
AEROPORTO DE DESTINO (REGIÃO)         object
AEROPORTO DE DESTINO (PAÍS)           object
AEROPORTO DE DESTINO (CONTINENTE)     object
NATUREZA                              object
GRUPO DE VOO                          object
PASSAGEIROS PAGOS                    float64
PASSAGEIROS GRÁTIS                   float64
CARGA PAGA (KG)                      float64
CARGA GRÁT

*As colunas do tipo 'object' referem-se a 'strings'*

**Selecionando os voos puramente brasileiros**

In [258]:
dados_anac_br = dados_anac.query( "`AEROPORTO DE ORIGEM (PAÍS)` == 'BRASIL' and `AEROPORTO DE DESTINO (PAÍS)` == 'BRASIL'" )

*Validando a seleção*

In [259]:
dados_anac_br[ ["AEROPORTO DE ORIGEM (PAÍS)", "AEROPORTO DE DESTINO (PAÍS)"] ].describe()

,AEROPORTO DE ORIGEM (PAÍS),AEROPORTO DE DESTINO (PAÍS)
count,311838,311838
unique,1,1
top,BRASIL,BRASIL
freq,311838,311838


## Agregando por par ORIGEM-DESTINO

Nesta etapa, construimos os pares únicos de aeroportos conectados por algum voo.

Os aeroportos serão os **nós** da nossa **rede**.
Dois aeroportos são conectados se existe ao menos um voo que os ligue.

Os pares finais vão conter:
  - Sigla, UF, Região e coordenadas de cada aeroporto
  - Total de voos entre eles
  - Total de voos feito por empresas brasileiras
  - Total de voos de natureza 'DOMÉSTICA'
  - Total de voos dos tipos *Improdutivo*, *Regulares* e *Não Regulares*  
  - Distância entre origem e destino

In [260]:
COLUMNS = ['AEROPORTO DE ORIGEM (SIGLA)',
           'AEROPORTO DE ORIGEM (UF)',
           'AEROPORTO DE ORIGEM (REGIÃO)',
           'AEROPORTO DE DESTINO (SIGLA)',
           'AEROPORTO DE DESTINO (UF)',
           'AEROPORTO DE DESTINO (REGIÃO)',
           'GRUPO DE VOO',
           'EMPRESA (NACIONALIDADE)',
           'DISTÂNCIA VOADA (KM)',
           'NATUREZA'
           ]

dados_anac_br = dados_anac_br[ COLUMNS ]

Criando variáveis auxiliares

In [261]:
dados_anac_br['VOO REGULAR'] = dados_anac_br['GRUPO DE VOO'] == 'REGULAR'
dados_anac_br['VOO NÃO REGULAR'] = dados_anac_br['GRUPO DE VOO'] == 'NÃO REGULAR'
dados_anac_br['IMPRODUTIVO'] = dados_anac_br['GRUPO DE VOO'] == 'IMPRODUTIVO'
dados_anac_br['EMPRESA NACIONAL'] = dados_anac_br['EMPRESA (NACIONALIDADE)'] == 'BRASILEIRA'
dados_anac_br['NATUREZA DOMÉSTICA'] =  dados_anac_br['NATUREZA'] == 'DOMÉSTICA'

dados_anac_br = dados_anac_br.drop( ['GRUPO DE VOO', 'EMPRESA (NACIONALIDADE)', 'NATUREZA'], axis=1 )

In [262]:
dados_gb = dados_anac_br.groupby( by=['AEROPORTO DE ORIGEM (SIGLA)', 'AEROPORTO DE DESTINO (SIGLA)'] )

In [263]:
dados_final = dados_gb[ ['AEROPORTO DE ORIGEM (UF)',
                         'AEROPORTO DE ORIGEM (REGIÃO)',
                         'AEROPORTO DE DESTINO (UF)',
                         'AEROPORTO DE DESTINO (REGIÃO)'] ].max()
dados_final[ 'DISTÂNCIA VOADA (KM)' ] = dados_gb[ 'DISTÂNCIA VOADA (KM)' ].max()
dados_final[ 'VOO REGULAR' ] = dados_gb[ 'VOO REGULAR' ].sum()
dados_final[ 'VOO NÃO REGULAR' ] = dados_gb[ 'VOO NÃO REGULAR' ].sum()
dados_final[ 'IMPRODUTIVO' ] = dados_gb[ 'IMPRODUTIVO' ].sum()
dados_final[ 'EMPRESA NACIONAL' ] = dados_gb[ 'EMPRESA NACIONAL' ].sum()
dados_final[ 'NATUREZA DOMÉSTICA' ] = dados_gb[ 'NATUREZA DOMÉSTICA' ].sum()
dados_final[ 'VOOS' ] = dados_final[ 'IMPRODUTIVO' ] + dados_final[ 'VOO REGULAR' ] + dados_final[ 'VOO NÃO REGULAR' ]

In [264]:
dados_final = dados_final.reset_index()

In [265]:
dados_final.head()

,AEROPORTO DE ORIGEM (SIGLA),AEROPORTO DE DESTINO (SIGLA),AEROPORTO DE ORIGEM (UF),AEROPORTO DE ORIGEM (REGIÃO),AEROPORTO DE DESTINO (UF),AEROPORTO DE DESTINO (REGIÃO),DISTÂNCIA VOADA (KM),VOO REGULAR,VOO NÃO REGULAR,IMPRODUTIVO,EMPRESA NACIONAL,NATUREZA DOMÉSTICA,VOOS
0,SBAA,SBAA,PA,NORTE,PA,NORTE,0.0,0,0,1,1,1,1
1,SBAA,SBBE,PA,NORTE,PA,NORTE,NaN,20,0,0,20,20,20
2,SBAA,SBCI,PA,NORTE,MA,NORDESTE,233.0,7,2,0,9,9,9
3,SBAA,SBGO,PA,NORTE,GO,CENTRO-OESTE,921.0,0,0,1,1,1,1
4,SBAA,SBMA,PA,NORTE,PA,NORTE,332.0,20,0,1,21,21,21


**Adicionando informações de coordenadas**

In [266]:
dados_coord = pd.read_csv('archive/airport-codes.csv')

In [267]:
dados_coord.columns

Index(['ident', 'type', 'name', 'elevation_ft', 'continent', 'iso_country',
       'iso_region', 'municipality', 'gps_code', 'iata_code', 'local_code',
       'coordinates'],
      dtype='object')

In [268]:
dados_coord = dados_coord[ ['gps_code', 'coordinates'] ]
dados_final = dados_final.merge( dados_coord, right_on = 'gps_code', left_on='AEROPORTO DE ORIGEM (SIGLA)' )
dados_final = dados_final.merge( dados_coord, right_on = 'gps_code', left_on='AEROPORTO DE DESTINO (SIGLA)' )

In [269]:
dados_final = dados_final.drop( ['gps_code_x', 'gps_code_y', 'coordinates_x'], axis=1 )
dados_final = dados_final.rename( columns={'coordinates_y':'COORDENADAS'} )

In [270]:
dados_final['LAT']  = dados_final['COORDENADAS'].apply( lambda x: float(x.split(',')[0]) )
dados_final['LONG']  = dados_final['COORDENADAS'].apply( lambda x: float(x.split(',')[1]) )
dados_final = dados_final.drop( ['COORDENADAS'], axis=1 )

**Renomeando colunas para caracteres ASCII e removendo espaços**

https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string

In [271]:
import unicodedata

def fix_columns_name(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn').replace(' ', '_').replace('(', '').replace(')', '')

In [272]:
dados_final = dados_final.rename( columns= { col:fix_columns_name(col) for col in dados_final.columns } )

**Removendo valores NaN**

In [273]:
print( f"Tamanho original - {len(dados_final)}" )
dados_final = dados_final.dropna()
print( f"Tamanho final - {len(dados_final)}" )

Tamanho original - 4594
Tamanho final - 4041


**Resultado Final**

In [274]:
dados_final.head()

,AEROPORTO_DE_ORIGEM_SIGLA,AEROPORTO_DE_DESTINO_SIGLA,AEROPORTO_DE_ORIGEM_UF,AEROPORTO_DE_ORIGEM_REGIAO,AEROPORTO_DE_DESTINO_UF,AEROPORTO_DE_DESTINO_REGIAO,DISTANCIA_VOADA_KM,VOO_REGULAR,VOO_NAO_REGULAR,IMPRODUTIVO,EMPRESA_NACIONAL,NATUREZA_DOMESTICA,VOOS,LAT,LONG
0,SBAA,SBAA,PA,NORTE,PA,NORTE,0.0,0,0,1,1,1,1,-49.301498,-8.34835
3,SBGO,SBAA,GO,CENTRO-OESTE,PA,NORTE,921.0,0,0,1,1,1,1,-49.301498,-8.34835
4,SBMA,SBAA,PA,NORTE,PA,NORTE,332.0,20,0,0,20,20,20,-49.301498,-8.34835
5,SBPJ,SBAA,TO,NORTE,PA,NORTE,4063.0,5,0,0,5,5,5,-49.301498,-8.34835
6,SJHG,SBAA,MT,CENTRO-OESTE,PA,NORTE,2130.0,0,0,7,7,7,7,-49.301498,-8.34835


## Construindo o Grafo com o NetworkX

A partir desse ponto, já podemos consturir nosso grafo com o NetworkX.

In [275]:
import networkx as nx
from pprint import pprint

In [276]:
graph_voos = nx.from_pandas_edgelist(dados_final,  
                                     source ='AEROPORTO_DE_ORIGEM_SIGLA',
                                     target ='AEROPORTO_DE_DESTINO_SIGLA',
                                     edge_attr = list(set(dados_final.columns) - {'AEROPORTO_DE_ORIGEM_SIGLA', 'AEROPORTO_DE_DESTINO_SIGLA'}),
                                     create_using = nx.DiGraph())

**Visualizando dados**

In [277]:
graph_voos.nodes

NodeView(('SBAA', 'SBGO', 'SBMA', 'SBPJ', 'SJHG', 'SNDC', 'SNKE', 'SWFX', 'SWGN', 'SBAR', 'SBBE', 'SBAT', 'SBBH', 'SBBR', 'SBBV', 'SBCF', 'SBCG', 'SBCJ', 'SBCT', 'SBCY', 'SBEG', 'SBFL', 'SBFZ', 'SBGL', 'SBGR', 'SBHT', 'SBIH', 'SBIZ', 'SBJP', 'SBKP', 'SBLO', 'SBMD', 'SBMK', 'SBMO', 'SBMQ', 'SBNT', 'SBOI', 'SBPA', 'SBPS', 'SBPV', 'SBQV', 'SBRB', 'SBRF', 'SBSG', 'SBSL', 'SBSN', 'SBSP', 'SBSV', 'SBTB', 'SBTE', 'SBTU', 'SBUL', 'SBVT', 'SDOW', 'SISO', 'SNEB', 'SNMZ', 'SNSW', 'SNVS', 'SNYA', 'SBCI', 'SBBW', 'SBCA', 'SBCH', 'SBCN', 'SBFI', 'SBIL', 'SBJD', 'SBJF', 'SBJI', 'SBJU', 'SBJV', 'SBMG', 'SBMT', 'SBNF', 'SBPL', 'SBRJ', 'SBRP', 'SBSJ', 'SBSR', 'SBTC', 'SBUR', 'SBVH', 'SDOV', 'SDUB', 'SJVO', 'SNBM', 'SNBR', 'SNGI', 'SNMH', 'SSKW', 'SWDB', 'SWGI', 'SWGP', 'SWHG', 'SWHP', 'SWIQ', 'SWLC', 'SWNH', 'SWNS', 'SWUA', 'SWVC', 'SWWU', 'SWZM', 'SJGU', 'SNFX', 'SBJC', 'SBAC', 'SBCB', 'SBJE', 'SBPB', 'SDSC', 'SDZG', 'SNAT', 'SNIG', 'SNOB', 'SNWS', 'SWBE', 'SBAE', 'SBAQ', 'SBAU', 'SBAX', 'SBBI', 'SBCM'

In [278]:
graph_voos.is_directed()

True

In [282]:
pprint(graph_voos['SBAA']['SNDC'])

{'AEROPORTO_DE_DESTINO_REGIAO': 'NORTE',
 'AEROPORTO_DE_DESTINO_UF': 'PA',
 'AEROPORTO_DE_ORIGEM_REGIAO': 'NORTE',
 'AEROPORTO_DE_ORIGEM_UF': 'PA',
 'DISTANCIA_VOADA_KM': 1909.0,
 'EMPRESA_NACIONAL': 31,
 'IMPRODUTIVO': 2,
 'LAT': -49.97990036010742,
 'LONG': -8.033289909362793,
 'NATUREZA_DOMESTICA': 31,
 'VOOS': 31,
 'VOO_NAO_REGULAR': 2,
 'VOO_REGULAR': 27}


## Salvando o grafo

**Salvando gml**

In [279]:
nx.write_gml(graph_voos, 'graph_br_flights.gml.gz')

**Testando arquivo**

In [280]:
g = nx.read_gml('graph_br_flights.gml.gz')

## Resumo final do grafo

Este notebook construiu um grafo Dirigido contendo informações de voos que ocorreram nos aeroportos brasileiros de 2000 a 2020.

Abaixo, uma lista das variáveis das arestas do grafo.

- **AEROPORTO_DE_ORIGEM_UF**
- **AEROPORTO_DE_ORIGEM_REGIAO**	
- **AEROPORTO_DE_DESTINO_UF**
- **AEROPORTO_DE_DESTINO_REGIAO**	
- **DISTANCIA_VOADA_KM**	: Distância entre origem e destino
- **VOOS** - Quantidade voos feitos entre os aeroportos
- **VOO_REGULAR**	: Quantidade de voos regulares feitas entre os aeroportos
- **VOO_NAO_REGULAR**	: Quantidade de voos não regulares feitas entre os aeroportos
- **IMPRODUTIVO**	: Quantidade de voos improdutivos feitas entre os aeroportos
- **EMPRESA_NACIONAL**	: Quantidade de voos de empresa nacional
- **NATUREZA_DOMESTICA**	: Quantidade de voos de natureza doméstica
- **LAT**	: Latitude
- **LONG** : Longitude